# 訓練貓狗圖片分類器 (使用 ResNet50)

以下是使用 PyTorch 在 Colab 上訓練 ResNet50 模型進行貓狗圖片分類的完整步驟。

## 1. 環境設置

首先，確保安裝必要的庫。

In [ ]:
# 安裝必要的庫
!pip install torch torchvision
!pip install albumentations

## 2. 資料集準備

### 2.1 檔案結構

根據新的資料集目錄結構，您的資料集位於 `dataset/` 資料夾內，包含 `training_dataset/` 和 `inference_dataset/`。其中，`training_dataset/` 包含訓練和驗證資料，`inference_dataset/` 則用於測試。

```
dataset/
├── inference_dataset/
│   ├── Cat/
│   └── Dog/
└── training_dataset/
    ├── Cat/
    └── Dog/
```

### 2.2 讀取資料集並進行預處理

由於 `training_dataset/` 只包含訓練資料，我們需要將其進一步劃分為訓練集和驗證集。以下是調整後的資料處理流程：

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

# 定義資料路徑
data_dir = 'dataset'

# 定義圖像轉換
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# 讀取訓練資料集
training_dataset = datasets.ImageFolder(os.path.join(data_dir, 'training_dataset'),
                                       data_transforms['train'])

# 讀取驗證資料集
validation_dataset = datasets.ImageFolder(os.path.join(data_dir, 'validation_dataset'),
                                         data_transforms['val'])

# 讀取測試資料集
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'inference_dataset'),
                                    data_transforms['test'])

# 建立 DataLoader
dataloaders = {
    'train': DataLoader(training_dataset, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(validation_dataset, batch_size=32, shuffle=False, num_workers=4),
    'test': DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
}

dataset_sizes = {
    'train': len(training_dataset),
    'val': len(validation_dataset),
    'test': len(test_dataset)
}

class_names = training_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 3. 模型構建

### 3.1 定義 ResNet50 模型並修改輸出層

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

def initialize_model(num_classes=2, feature_extract=True):
    model = models.resnet50(pretrained=True)
    if feature_extract:
        for param in model.parameters():
            param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, num_classes),
        nn.Softmax(dim=1)
    )
    return model

model = initialize_model(num_classes=2, feature_extract=True)
model = model.to(device)


## 4. 訓練函數

In [ ]:
import time
import copy

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # 每個 epoch 都有訓練和驗證階段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # 迭代資料
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 前向傳播
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 反向傳播 + 優化
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # 統計
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 深度拷貝模型
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {int(time_elapsed//60)}m {int(time_elapsed%60)}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # 載入最佳模型權重
    model.load_state_dict(best_model_wts)
    return model

## 5. 訓練模型

### 5.1 不使用 Random Erasing

In [ ]:
import torch.optim as optim

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# 訓練模型
model_no_erasing = initialize_model()
model_no_erasing = model_no_erasing.to(device)

model_no_erasing = train_model(model_no_erasing, dataloaders, criterion, optimizer, num_epochs=25)

# 儲存模型
torch.save(model_no_erasing.state_dict(), 'resnet50_no_erasing.pth')

### 5.2 使用 Random Erasing 進行資料增強

In [ ]:
from torchvision import transforms
import numpy as np
from PIL import Image

# 自定義轉換，以整合隨機擦除
class RandomErasingTransform:
    def __init__(self, probability=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random'):
        self.random_erasing = transforms.RandomErasing(p=probability, scale=scale, ratio=ratio, value=value)

    def __call__(self, img):
        img = np.array(img)
        img = Image.fromarray(img)
        img = self.random_erasing(img)
        return img

# 更新資料轉換
data_transforms_erasing = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        RandomErasingTransform(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': data_transforms['val'],
    'test': data_transforms['test']
}

# 讀取訓練資料集
training_dataset_erasing = datasets.ImageFolder(os.path.join(data_dir, 'training_dataset'),
                                               data_transforms_erasing['train'])

# 讀取驗證資料集
validation_dataset_erasing = datasets.ImageFolder(os.path.join(data_dir, 'validation_dataset'),
                                                 data_transforms_erasing['val'])

# 讀取測試資料集（保持不變）
test_dataset_erasing = test_dataset

# 建立 DataLoader
dataloaders_erasing = {
    'train': DataLoader(training_dataset_erasing, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(validation_dataset_erasing, batch_size=32, shuffle=False, num_workers=4),
    'test': DataLoader(test_dataset_erasing, batch_size=32, shuffle=False, num_workers=4)
}

# 定義模型
model_erasing = initialize_model()
model_erasing = model_erasing.to(device)

# 定義優化器
optimizer_erasing = optim.Adam(model_erasing.fc.parameters(), lr=0.001)

# 訓練模型
model_erasing = train_model(model_erasing, dataloaders_erasing, criterion, optimizer_erasing, num_epochs=25)

# 儲存模型
torch.save(model_erasing.state_dict(), 'resnet50_with_erasing.pth')

## 6. 驗證模型準確度

In [ ]:
def evaluate_model(model, dataloader, phase='val'):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloader[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / dataset_sizes[phase]
    return accuracy.item()

acc_no_erasing = evaluate_model(model_no_erasing, dataloaders, phase='val')
acc_with_erasing = evaluate_model(model_erasing, dataloaders_erasing, phase='val')

print(f'驗證準確度 (無 Random Erasing): {acc_no_erasing:.4f}')
print(f'驗證準確度 (使用 Random Erasing): {acc_with_erasing:.4f}')

### 測試集評估（可選）

若您希望在測試集上評估模型表現，可以新增以下程式碼：

acc_test_no_erasing = evaluate_model(model_no_erasing, dataloaders, phase='test')
acc_test_with_erasing = evaluate_model(model_erasing, dataloaders_erasing, phase='test')

print(f'測試準確度 (無 Random Erasing): {acc_test_no_erasing:.4f}')
print(f'測試準確度 (使用 Random Erasing): {acc_test_with_erasing:.4f}')

### 測試集評估（可選）

若您希望在測試集上評估模型表現，可以新增以下程式碼：

In [ ]:
acc_test_no_erasing = evaluate_model(model_no_erasing, dataloaders, phase='test')
acc_test_with_erasing = evaluate_model(model_erasing, dataloaders_erasing, phase='test')

print(f'測試準確度 (無 Random Erasing): {acc_test_no_erasing:.4f}')
print(f'測試準確度 (使用 Random Erasing): {acc_test_with_erasing:.4f}')

## 7. 儲存最佳模型

In [ ]:
# 若需匯出到 Google Drive，可使用以下指令

from google.colab import drive
drive.mount('/content/drive')

# 儲存到指定路徑
torch.save(model_no_erasing.state_dict(), '/content/drive/MyDrive/resnet50_no_erasing.pth')
torch.save(model_erasing.state_dict(), '/content/drive/MyDrive/resnet50_with_erasing.pth')

## 8. 完整程式碼總結

將上述所有程式碼片段整合到 `colab_train2.ipynb` 中，即可完成訓練貓狗分類器的流程。請確保資料集位於正確的路徑 `dataset/`，並根據需要調整超參數。

## 參考資料

- [PyTorch 官方文檔](https://pytorch.org/docs/stable/index.html)
- [Albumentations 資料增強庫](https://albumentations.ai/docs/)
- [ResNet 論文](https://arxiv.org/abs/1512.03385)

## 結論

根據新的資料集目錄結構，您已成功調整程式碼以正確讀取和處理資料。透過上述步驟，您可以訓練兩個使用 ResNet50 的貓狗圖片分類器，一個使用了 Random Erasing 資料增強，另一個則沒有。最終，您將能夠比較兩者在驗證集上的表現差異，並選擇最佳模型進行部署或進一步優化。

# 祝您訓練順利！